In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('level2_i_20221201.csv')
df.head()

long = 200
short = 100


a = df['midp'].diff(long) / df['midp'].shift(long)
a = a.dropna()
a.reset_index(drop = True, inplace = True)
df['long return'] = a

b = df['midp'].diff(short) / df['midp'].shift(short)
b = b.dropna()
b.reset_index(drop = True, inplace = True)
df['short return'] = b

# only keep the factors
col_reserved = ['fac{}'.format(i) for i in range(1, 34)]
col_reserved.append('long return')
col_reserved.append('short return')

new_df = df[col_reserved]
linear_df = new_df[1200:-1200]
#linear_df.fillna(0, inplace = True)
linear_df.reset_index(drop = True, inplace = True)
linear_df

,fac1,fac2,fac3,fac4,fac5,fac6,fac7,fac8,fac9,fac10,...,fac26,fac27,fac28,fac29,fac30,fac31,fac32,fac33,long return,short return
0,-0.000214,-0.019506,-0.555555,-0.044372,-0.006845,0.012014,0.017698,0.005543,-0.005437,3.0,...,0.0,-0.028571,-0.002507,0.000537,0.000537,0.000493,0.000235,0.000121,0.0,0.000647
1,-0.000292,-0.132730,-0.830189,-0.068396,-0.011363,0.008915,0.016692,0.004849,-0.005456,2.0,...,0.0,-0.048544,-0.002496,0.000523,0.000523,0.000471,0.000226,0.000183,0.0,0.000647
2,-0.000296,-0.017991,-0.999999,-0.067358,-0.010600,0.007809,0.016580,0.004801,-0.005237,0.0,...,0.0,-0.039216,-0.002493,0.000523,0.000523,0.000468,0.000250,0.000201,0.0,0.000647
3,-0.000290,-0.021552,-0.517241,-0.046512,-0.010224,0.007456,0.016282,0.004631,-0.005729,0.0,...,0.0,-0.049505,-0.002485,0.000524,0.000524,0.000468,0.000250,0.000234,0.0,0.000647
4,-0.000322,-0.055639,-1.000000,-0.056485,-0.011385,0.006608,0.015556,0.004362,-0.005393,-1.0,...,0.0,-0.070707,-0.002460,0.000521,0.000521,0.000463,0.000249,0.000224,0.0,0.000647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74345,-0.000054,0.001085,0.333332,0.000816,-0.002180,0.000699,0.004050,-0.004290,-0.006255,0.0,...,-0.0,-0.333333,-0.000835,0.000040,0.000040,0.000171,0.000224,0.000326,0.0,0.000000
74346,-0.000054,-0.001086,-0.999990,0.000815,-0.002074,0.000458,0.004056,-0.004131,-0.006255,0.0,...,-0.0,-0.333333,-0.000835,0.000044,0.000044,0.000171,0.000223,0.000326,0.0,0.000000
74347,-0.000054,-0.001085,-0.999990,0.000271,-0.001803,0.000459,0.004125,-0.003815,-0.006258,0.0,...,-0.0,-0.333333,-0.000835,0.000053,0.000053,0.000171,0.000223,0.000326,0.0,0.000000
74348,-0.000053,0.004329,0.999998,0.000813,0.000382,0.007821,0.004176,-0.002501,-0.006255,1.0,...,0.0,-0.333333,-0.000835,0.000134,0.000134,0.000169,0.000036,0.000109,0.0,0.000000


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

window_size = 500

predict_long = []
predict_short = []
# this sequence record the R square of validation set
R_square_long = []
R_square_short = []

#limit = len(linear_df)
limit = 10000

for i in range(window_size, limit):
    train_X = linear_df.iloc[i - window_size : i - 200, :-2]
    train_y_long = linear_df.iloc[i - window_size : i - 200, -2]
    train_y_short = linear_df.iloc[i - window_size : i - 200, -1]

    train_X_long, validate_X_long, train_y_long, validate_y_long = train_test_split(train_X, train_y_long, test_size = 0.1, random_state = 11)
    train_X_short, validate_X_short, train_y_short, validate_y_short = train_test_split(train_X, train_y_short, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_long = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_short = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    model_short = RandomForestRegressor(n_estimators = 15, max_depth = 5, random_state = 11)
    model_long = RandomForestRegressor(n_estimators = 15, max_depth = 5, random_state = 11)

    model_short.fit(train_X_short, train_y_short)
    model_long.fit(train_X_long, train_y_long)

    predict_short.append(model_short.predict(test_X))
    predict_long.append(model_long.predict(test_X))

    R_square_short.append(model_short.score(validate_X_short, validate_y_short))
    R_square_long.append(model_long.score(validate_X_long, validate_y_long))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_short = {}'.format(model_short.score(validate_X_short, validate_y_short)))
        print('R_square_long = {}'.format(model_long.score(validate_X_long, validate_y_long)))

i = 1000
R_square_short = 0.991242062937933
R_square_long = 0.9841268368717601
i = 2000
R_square_short = 0.964156130077874
R_square_long = 0.8949109382834024
i = 3000
R_square_short = 0.8366978020819843
R_square_long = 0.9920138652699815
i = 4000
R_square_short = 0.9996661710642433
R_square_long = 0.9198225472932364
i = 5000
R_square_short = 0.985647529211422
R_square_long = 0.9705116372838181
i = 6000
R_square_short = 0.9977868206968679
R_square_long = 0.8607690019023397
i = 7000
R_square_short = 0.8396403581335539
R_square_long = 0.9409233452081704
i = 8000
R_square_short = 0.5389696215572051
R_square_long = 0.8810354253387697
i = 9000
R_square_short = 0.9702476523665432
R_square_long = 0.9429126153707877


In [7]:
predict_short = np.array(predict_short).reshape(-1)
predict_long = np.array(predict_long).reshape(-1)
R_square_short = np.array(R_square_short).reshape(-1)
R_square_long = np.array(R_square_long).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_short'] = predict_short
new_df['predict_long'] = predict_long
new_df['R_square_short'] = R_square_short
new_df['R_square_long'] = R_square_long

new_df['long return'] = linear_df['long return'].iloc[window_size:limit].reset_index(drop = True)
new_df['short return'] = linear_df['short return'].iloc[window_size:limit].reset_index(drop = True)
new_df['predict_long_sign'] = new_df['predict_long'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df['predict_short_sign'] = new_df['predict_short'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df

,predict_short,predict_long,R_square_short,R_square_long,long return,short return,predict_long_sign,predict_short_sign
0,0.000281,-0.000647,0.987859,0.966375,0.000000,0.000000,-1,1
1,0.000302,-0.000638,0.978538,0.983616,0.000000,0.000000,-1,1
2,0.000130,-0.000791,0.997654,0.966826,0.000000,0.000000,-1,1
3,0.000130,-0.000641,0.997213,0.976545,0.000000,0.000000,-1,1
4,0.000022,-0.000615,0.992690,0.972963,0.000000,0.000000,-1,1
...,...,...,...,...,...,...,...,...
9495,-0.000301,-0.000733,0.888510,0.893290,0.000648,0.000648,-1,-1
9496,-0.000058,-0.000820,0.868515,0.775323,0.000648,0.000648,-1,-1
9497,0.000040,-0.001078,0.767409,0.735676,0.000648,0.000648,-1,1
9498,-0.000228,-0.000906,0.602362,0.873678,0.000648,0.000648,-1,-1


In [8]:
print(new_df[['predict_short_sign', 'short return']].corr())
print(new_df[['predict_long_sign', 'long return']].corr())

print(new_df[['predict_short', 'short return']].corr())
print(new_df[['predict_long', 'long return']].corr())

                    predict_short_sign  short return
predict_short_sign            1.000000      0.123849
short return                  0.123849      1.000000
                   predict_long_sign  long return
predict_long_sign           1.000000     0.173581
long return                 0.173581     1.000000
               predict_short  short return
predict_short       1.000000      0.064968
short return        0.064968      1.000000
              predict_long  long return
predict_long      1.000000     0.061095
long return       0.061095     1.000000


In [9]:
predict_long = []
predict_short = []
# this sequence record the R square of validation set
R_square_long = []
R_square_short = []

for i in range(window_size, limit):
    train_X = linear_df.iloc[i - window_size : i - 200, :-2]
    train_y_long = linear_df.iloc[i - window_size : i - 200, -2]
    train_y_short = linear_df.iloc[i - window_size : i - 200, -1]

    train_X_long, validate_X_long, train_y_long, validate_y_long = train_test_split(train_X, train_y_long, test_size = 0.1, random_state = 11)
    train_X_short, validate_X_short, train_y_short, validate_y_short = train_test_split(train_X, train_y_short, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_long = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_short = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    model_short = GradientBoostingRegressor(n_estimators = 15, max_depth = 3, learning_rate = 0.1)
    model_long = GradientBoostingRegressor(n_estimators = 15, max_depth = 3, learning_rate = 0.1)

    model_short.fit(train_X_short, train_y_short)
    model_long.fit(train_X_long, train_y_long)

    predict_short.append(model_short.predict(test_X))
    predict_long.append(model_long.predict(test_X))

    R_square_short.append(model_short.score(validate_X_short, validate_y_short))
    R_square_long.append(model_long.score(validate_X_long, validate_y_long))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_short = {}'.format(model_short.score(validate_X_short, validate_y_short)))
        print('R_square_long = {}'.format(model_long.score(validate_X_long, validate_y_long)))

i = 1000
R_square_short = 0.9189060974516828
R_square_long = 0.9256362944043133
i = 2000
R_square_short = 0.951040993606766
R_square_long = 0.8740299085346765
i = 3000
R_square_short = 0.8392495027700955
R_square_long = 0.9573826175558484
i = 4000
R_square_short = 0.9566680366580629
R_square_long = 0.8992223759523994
i = 5000
R_square_short = 0.9180331227026699
R_square_long = 0.9557400771792416
i = 6000
R_square_short = 0.9394799033032125
R_square_long = 0.8139439966908292
i = 7000
R_square_short = 0.8180986452437994
R_square_long = 0.8586343901874594
i = 8000
R_square_short = 0.5560029585408042
R_square_long = 0.817840840320063
i = 9000
R_square_short = 0.9111686847016195
R_square_long = 0.9188964578502823


In [10]:
predict_short = np.array(predict_short).reshape(-1)
predict_long = np.array(predict_long).reshape(-1)
R_square_short = np.array(R_square_short).reshape(-1)
R_square_long = np.array(R_square_long).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_short'] = predict_short
new_df['predict_long'] = predict_long
new_df['R_square_short'] = R_square_short
new_df['R_square_long'] = R_square_long

new_df['long return'] = linear_df['long return'].iloc[window_size:limit].reset_index(drop = True)
new_df['short return'] = linear_df['short return'].iloc[window_size:limit].reset_index(drop = True)

new_df['predict_long_sign'] = new_df['predict_long'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df['predict_short_sign'] = new_df['predict_short'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df

,predict_short,predict_long,R_square_short,R_square_long,long return,short return,predict_long_sign,predict_short_sign
0,-0.000117,-0.000484,0.938942,0.926532,0.000000,0.000000,-1,-1
1,-0.000093,-0.000463,0.939413,0.948563,0.000000,0.000000,-1,-1
2,-0.000143,-0.000449,0.932038,0.949287,0.000000,0.000000,-1,-1
3,-0.000198,-0.000489,0.936552,0.948067,0.000000,0.000000,-1,-1
4,-0.000203,-0.000456,0.927913,0.944699,0.000000,0.000000,-1,-1
...,...,...,...,...,...,...,...,...
9495,0.000003,-0.000582,0.776031,0.813562,0.000648,0.000648,-1,1
9496,-0.000039,-0.000693,0.771947,0.680610,0.000648,0.000648,-1,-1
9497,-0.000017,-0.000740,0.708254,0.653359,0.000648,0.000648,-1,-1
9498,-0.000073,-0.000824,0.522435,0.773731,0.000648,0.000648,-1,-1


In [11]:
print(new_df[['predict_short_sign', 'short return']].corr())
print(new_df[['predict_long_sign', 'long return']].corr())

                    predict_short_sign  short return
predict_short_sign            1.000000      0.141955
short return                  0.141955      1.000000
                   predict_long_sign  long return
predict_long_sign           1.000000     0.174338
long return                 0.174338     1.000000
